# MODELO MOTIVO RECHAZO

In [60]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

## --- Conexión a la base de datos ---

In [61]:
db_file = 'data/vacaciones_empresa.db'
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

## --- Carga de datos desde las tablas ---

In [62]:
empleados_df = pd.read_sql_query("SELECT * FROM empleados", conn)
solicitudes_df = pd.read_sql_query("SELECT * FROM solicitudes_vacaciones", conn)
calendario_df = pd.read_sql_query("SELECT * FROM calendario_laboral", conn)

## --- Fusión de DataFrames ---

In [63]:
# Fusionar solicitudes con información del empleado
rechazadas_df = solicitudes_df[solicitudes_df['estado'] == 'Rechazada'].copy()
rechazadas_df = pd.merge(rechazadas_df, empleados_df, on='id_empleado', suffixes=('_solicitud', '_empleado'))

In [64]:
# Fusionar con información del calendario (opcional)
rechazadas_df = pd.merge(rechazadas_df, calendario_df, left_on='fecha_inicio', right_on='fecha', how='left')
rechazadas_df.drop(columns=['fecha'], inplace=True)

## --- Preprocesamiento de datos para motivos de rechazo ---

In [65]:
# 1. Manejo de valores faltantes (podría haber algunos en otras columnas)
rechazadas_df.fillna('', inplace=True)

In [66]:
# 2. Creación de características (similar al modelo de aceptación/rechazo)
rechazadas_df['duracion_dias'] = rechazadas_df['duracion_dias'].astype(int)
rechazadas_df['antiguedad_meses'] = rechazadas_df['antiguedad_meses'].astype(int)
rechazadas_df['mes_solicitud'] = rechazadas_df['mes_solicitud'].astype(int)

# Convertir columnas de fecha a datetime
rechazadas_df['fecha_contratacion'] = pd.to_datetime(rechazadas_df['fecha_contratacion'])
rechazadas_df['fecha_inicio'] = pd.to_datetime(rechazadas_df['fecha_inicio'])
rechazadas_df['fecha_fin'] = pd.to_datetime(rechazadas_df['fecha_fin'])
rechazadas_df['fecha_solicitud'] = pd.to_datetime(rechazadas_df['fecha_solicitud'])

# Calcular la diferencia en días entre la fecha de solicitud y la fecha de inicio
rechazadas_df['dias_anticipacion'] = (rechazadas_df['fecha_inicio'] - rechazadas_df['fecha_solicitud']).dt.days

# Extraer información temporal adicional
rechazadas_df['dia_semana_inicio'] = rechazadas_df['fecha_inicio'].dt.day_name()
rechazadas_df['mes_inicio'] = rechazadas_df['fecha_inicio'].dt.month

In [67]:
# 3. Codificación de variables categóricas
categorical_cols = ['departamento', 'puesto', 'dia_semana_solicitud', 'genero', 'nivel_educacion', 'dia_semana', 'temporada', 'dia_semana_inicio']
label_encoders_motivo = {}
for col in categorical_cols:
    if col in rechazadas_df.columns:
        label_encoders_motivo[col] = LabelEncoder()
        rechazadas_df[col] = label_encoders_motivo[col].fit_transform(rechazadas_df[col].astype(str))
    else:
        print(f"Advertencia (Motivo): La columna categórica '{col}' no se encontró en el DataFrame de rechazadas.")

In [68]:
# 4. Codificación de la variable objetivo 'motivo_rechazo'
label_encoder_motivo_rechazo = LabelEncoder()
rechazadas_df['motivo_rechazo_encoded'] = label_encoder_motivo_rechazo.fit_transform(rechazadas_df['motivo_rechazo'])
num_clases_motivo = len(label_encoder_motivo_rechazo.classes_)
print(f"Número de clases de motivos de rechazo (antes de dividir): {num_clases_motivo}")
print("Clases de motivos de rechazo (antes de dividir):", label_encoder_motivo_rechazo.classes_)

Número de clases de motivos de rechazo (antes de dividir): 11
Clases de motivos de rechazo (antes de dividir): ['Duración de vacaciones excede el límite permitido de 10 días.'
 'Duración de vacaciones excede el límite permitido de 12 días.'
 'Duración de vacaciones excede el límite permitido de 15 días.'
 'Duración de vacaciones excede el límite permitido de 18 días.'
 'No se cumple la antigüedad mínima de 12.0 meses.'
 'No se cumple la antigüedad mínima de 3.0 meses.'
 'No se cumple la antigüedad mínima de 6.0 meses.'
 'Se supera el número máximo de 1.0 empleados de vacaciones simultáneamente en el departamento.'
 'Se supera el número máximo de 2.0 empleados de vacaciones simultáneamente en el departamento.'
 'Se supera el número máximo de 3.0 empleados de vacaciones simultáneamente en el departamento.'
 'Solicitud dentro del periodo de bloqueo de vacaciones.']


In [69]:
# 5. Normalización de variables numéricas
numerical_cols = ['antiguedad_meses', 'edad', 'duracion_dias', 'periodo_anio', 'mes_solicitud', 'dias_anticipacion', 'mes_inicio']
scaler_motivo = StandardScaler()
numerical_cols_present_motivo = [col for col in numerical_cols if col in rechazadas_df.columns]
if numerical_cols_present_motivo:
    rechazadas_df[numerical_cols_present_motivo] = scaler_motivo.fit_transform(rechazadas_df[numerical_cols_present_motivo])
else:
    print("Advertencia (Motivo): No se encontraron columnas numéricas para normalizar en el DataFrame de rechazadas.")

## --- Preparación de los datos para el modelo de clasificación (Motivo de Rechazo) ---

In [70]:
X_motivo = rechazadas_df.drop(columns=['id_solicitud', 'id_empleado', 'nombre', 'fecha_contratacion', 'fecha_inicio', 'fecha_fin', 'fecha_solicitud', 'estado', 'motivo_rechazo', 'motivo_rechazo_encoded', 'correo_electronico'])
y_motivo = rechazadas_df['motivo_rechazo_encoded']

## --- División de datos en entrenamiento y prueba ---

In [71]:
X_train_motivo, X_test_motivo, y_train_motivo, y_test_motivo = train_test_split(X_motivo, y_motivo, test_size=0.2, random_state=42, stratify=y_motivo)

## --- Entrenamiento del modelo de clasificación (Random Forest) ---

In [72]:
model_motivo_rechazo = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
model_motivo_rechazo.fit(X_train_motivo, y_train_motivo)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

## --- Evaluación del modelo de clasificación ---

In [73]:
y_pred_motivo = model_motivo_rechazo.predict(X_test_motivo)
accuracy_motivo = accuracy_score(y_test_motivo, y_pred_motivo)

# Obtener las clases presentes en el conjunto de prueba
clases_presentes_test = np.unique(y_test_motivo)

# Obtener los nombres de las clases correspondientes usando el encoder original
target_names_actual = label_encoder_motivo_rechazo.inverse_transform(clases_presentes_test)

report_motivo = classification_report(y_test_motivo, y_pred_motivo, target_names=target_names_actual)

print("\n--- Resultados del Modelo de Predicción del Motivo de Rechazo ---")
print(f"Precisión del modelo (motivo de rechazo): {accuracy_motivo:.4f}")
print("Reporte de Clasificación (motivo de rechazo):\n", report_motivo)



--- Resultados del Modelo de Predicción del Motivo de Rechazo ---
Precisión del modelo (motivo de rechazo): 0.8362
Reporte de Clasificación (motivo de rechazo):
                                                                                                precision    recall  f1-score   support

                                Duración de vacaciones excede el límite permitido de 10 días.       0.89      0.89      0.89       244
                                Duración de vacaciones excede el límite permitido de 12 días.       0.73      0.87      0.79       120
                                Duración de vacaciones excede el límite permitido de 15 días.       0.69      0.79      0.74        67
                                Duración de vacaciones excede el límite permitido de 18 días.       0.89      0.89      0.89        19
                                             No se cumple la antigüedad mínima de 12.0 meses.       0.50      0.51      0.51        35
                          

## --- Cierre de la conexión a la base de datos ---

In [74]:
conn.close()